<a href="https://www.kaggle.com/code/waheedhussain2/mnist-k-fold-resnet-cnn-acc-99-43?scriptVersionId=206896993" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# introduction

The MNIST digit recognition competition on Kaggle challenges participants to classify handwritten digits (0-9) from grayscale images. It’s a well-known beginner-friendly competition, ideal for exploring image classification techniques and experimenting with deep learning models. Participants use a dataset of labeled training images to build models that can accurately predict the labels for a set of unlabeled test images. The competition is a great entry point for those interested in computer vision, allowing them to apply neural networks and other machine learning techniques while competing against a global community of learners and experts.

# Hyperparameters and Techniques
**Convolutional Filters:** 64, 128, and 256 filters across different layers.

**Dropout Rates:** Increasing dropout rates (0.2, 0.3, 0.4, and 0.5) for better regularization.

**Batch Size:** 256, chosen to balance training speed and stability.

**Optimizer:** Adam with an initial learning rate of 0.001, which works well with CNNs for fast convergence.

**Cross-Validation:** 5-fold to improve robustness.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
train = '/kaggle/input/digit-recognizer/train.csv'

In [4]:
train_data= pd.read_csv(train)

In [5]:
train_data.shape

(42000, 785)

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from sklearn.model_selection import train_test_split, KFold
import pandas as pd
import numpy as np


In [7]:
# Load the data
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
X = data.iloc[:, 1:].values.reshape(-1, 28, 28, 1) / 255.0  # Normalize pixel values
y = data.iloc[:, 0].values

# Train/Validation Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
# Model architecture with ResNet blocks for better accuracy
def build_model():
    model = models.Sequential()
    
    # Initial Convolution layer
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    
    # Residual block 1
    for _ in range(2):
        model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
        model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.3))
    
    # Residual block 2
    for _ in range(2):
        model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
        model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.4))
    
    # Fully connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))
    
    return model


In [9]:
"""
# Training function with cross-validation and learning rate scheduler
def train_and_evaluate(X_train, y_train):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train)):
        print(f"Training fold {fold+1}...")
        
        model = build_model()
        model.compile(optimizer=optimizers.Adam(learning_rate=0.01),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        
        lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, verbose=1)
        early_stopping = callbacks.EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True)
        
        history = model.fit(X_train[train_idx], y_train[train_idx],
                            validation_data=(X_train[val_idx], y_train[val_idx]),
                            epochs=20,
                            batch_size=256,
                            callbacks=[lr_scheduler, early_stopping],
                            verbose=2)
        
        score = model.evaluate(X_train[val_idx], y_train[val_idx], verbose=0)
        print(f"Fold {fold+1} - Validation accuracy: {score[1]}")
        scores.append(score[1])
        
    print(f"Mean cross-validated accuracy: {np.mean(scores)}")
    return model

# Train and evaluate
final_model1 = train_and_evaluate(X_train, y_train)

# Evaluate on validation set
val_loss, val_accuracy = final_model1.evaluate(X_val, y_val, verbose=0)
print(f"Final model validation accuracy: {val_accuracy}")
"""

'\n# Training function with cross-validation and learning rate scheduler\ndef train_and_evaluate(X_train, y_train):\n    kfold = KFold(n_splits=5, shuffle=True, random_state=42)\n    scores = []\n    \n    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train)):\n        print(f"Training fold {fold+1}...")\n        \n        model = build_model()\n        model.compile(optimizer=optimizers.Adam(learning_rate=0.01),\n                      loss=\'sparse_categorical_crossentropy\',\n                      metrics=[\'accuracy\'])\n        \n        lr_scheduler = callbacks.ReduceLROnPlateau(monitor=\'val_loss\', factor=0.5, patience=8, verbose=1)\n        early_stopping = callbacks.EarlyStopping(monitor=\'val_accuracy\', patience=8, restore_best_weights=True)\n        \n        history = model.fit(X_train[train_idx], y_train[train_idx],\n                            validation_data=(X_train[val_idx], y_train[val_idx]),\n                            epochs=20,\n                      

In [10]:
# Evaluate on validation set
#val_loss, val_accuracy = final_model1.evaluate(X_val, y_val, verbose=0)
#print(f"Final model validation accuracy: {val_accuracy}")


In [11]:

# Training function with cross-validation and learning rate scheduler
def train_and_evaluate(X_train, y_train):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train)):
        print(f"Training fold {fold+1}...")
        
        model = build_model()
        model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        
        lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, verbose=1)
        early_stopping = callbacks.EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True)
        
        history = model.fit(X_train[train_idx], y_train[train_idx],
                            validation_data=(X_train[val_idx], y_train[val_idx]),
                            epochs=25,
                            batch_size=256,
                            callbacks=[lr_scheduler, early_stopping],
                            verbose=2)
        
        score = model.evaluate(X_train[val_idx], y_train[val_idx], verbose=0)
        print(f"Fold {fold+1} - Validation accuracy: {score[1]}")
        scores.append(score[1])
        
    print(f"Mean cross-validated accuracy: {np.mean(scores)}")
    return model

# Train and evaluate
final_model = train_and_evaluate(X_train, y_train)

# Evaluate on validation set
val_loss, val_accuracy = final_model.evaluate(X_val, y_val, verbose=0)
print(f"Final model validation accuracy: {val_accuracy}")


Training fold 1...
Epoch 1/25


I0000 00:00:1731446617.044981      66 service.cc:145] XLA service 0x7ae63c0053d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731446617.045033      66 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1731446626.386487      66 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


105/105 - 16s - 155ms/step - accuracy: 0.9042 - loss: 0.3279 - val_accuracy: 0.1126 - val_loss: 11.3489 - learning_rate: 0.0010
Epoch 2/25
105/105 - 2s - 18ms/step - accuracy: 0.9745 - loss: 0.0814 - val_accuracy: 0.1126 - val_loss: 12.4689 - learning_rate: 0.0010
Epoch 3/25
105/105 - 2s - 18ms/step - accuracy: 0.9831 - loss: 0.0534 - val_accuracy: 0.1126 - val_loss: 8.3651 - learning_rate: 0.0010
Epoch 4/25
105/105 - 2s - 18ms/step - accuracy: 0.9856 - loss: 0.0460 - val_accuracy: 0.3519 - val_loss: 2.7799 - learning_rate: 0.0010
Epoch 5/25
105/105 - 2s - 18ms/step - accuracy: 0.9878 - loss: 0.0361 - val_accuracy: 0.9338 - val_loss: 0.2187 - learning_rate: 0.0010
Epoch 6/25
105/105 - 2s - 18ms/step - accuracy: 0.9888 - loss: 0.0350 - val_accuracy: 0.9817 - val_loss: 0.0577 - learning_rate: 0.0010
Epoch 7/25
105/105 - 2s - 18ms/step - accuracy: 0.9915 - loss: 0.0279 - val_accuracy: 0.9900 - val_loss: 0.0365 - learning_rate: 0.0010
Epoch 8/25
105/105 - 2s - 18ms/step - accuracy: 0.9916 

In [12]:
# Evaluate on validation set
val_loss, val_accuracy = final_model.evaluate(X_val, y_val, verbose=0)
print(f"Final model validation accuracy: {val_accuracy}")


Final model validation accuracy: 0.994523823261261


In [13]:
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')


In [14]:
test_data.shape

(28000, 784)

In [15]:
#sample_sub = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')


In [16]:
#sample_sub.head()

In [17]:
# Load the test data and preprocess it
X_test = test_data.values.reshape(-1, 28, 28, 1) / 255.0  # Normalize and reshape


In [18]:
# Predict using the trained model
predictions = final_model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

875/875 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [19]:
# Prepare the output DataFrame in the required format
output = pd.DataFrame({
    'ImageId': np.arange(1, len(predicted_labels) + 1),
    'Label': predicted_labels
})

In [20]:
# Save to CSV
output.to_csv('submission99-428.csv', index=False)

# Display first few rows of the submission file
output.head()


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


# Updated model architecture with deeper ResNet blocks
def build_model():
    model = models.Sequential()
    
    # Initial Convolution layer
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    
    # Residual block 1 with increased depth
    for _ in range(3):
        model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
        model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.3))
    
    # Residual block 2 with increased depth
    for _ in range(3):
        model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
        model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.4))
    
    # Fully connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(256, activation='relu'))  # New dense layer
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))  # Additional dropout layer
    model.add(layers.Dense(10, activation='softmax'))
    
    return model

# Training function with adjusted scheduler and early stopping
def train_and_evaluate(X_train, y_train):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train)):
        print(f"Training fold {fold+1}...")
        
        model = build_model()
        model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        
        lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=1e-6, verbose=1)
        early_stopping = callbacks.EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True)
        
        history = model.fit(X_train[train_idx], y_train[train_idx],
                            validation_data=(X_train[val_idx], y_train[val_idx]),
                            epochs=30,
                            batch_size=256,
                            callbacks=[lr_scheduler, early_stopping],
                            verbose=2)
        
        score = model.evaluate(X_train[val_idx], y_train[val_idx], verbose=0)
        print(f"Fold {fold+1} - Validation accuracy: {score[1]}")
        scores.append(score[1])
        
    print(f"Mean cross-validated accuracy: {np.mean(scores)}")
    return model

# Train and evaluate with updated model
final_modelL = train_and_evaluate(X_train, y_train)

# Evaluate on validation set
val_loss, val_accuracy = final_modelL.evaluate(X_val, y_val, verbose=0)
print(f"Final model validation accuracy: {val_accuracy}")
